In [2]:
using BenchmarkTools

In [210]:
include("src/staticvars.jl")

IJulia.sp

In [211]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

y

In [212]:
x + y

y + x

In [213]:
p = 3x + 2y

2y + 3x

In [214]:
x^2 + y + x * y

y + xy + x^2

In [215]:
p = x^2 + y + x * x + 3 * x * y + x * y^2

3xy + xy^2 + x^2 + y + x^2

In [208]:
@benchmark $p + $x

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  2
  --------------
  minimum time:     73.937 ns (0.00% GC)
  median time:      80.489 ns (0.00% GC)
  mean time:        101.192 ns (16.47% GC)
  maximum time:     2.523 μs (91.66% GC)
  --------------
  samples:          10000
  evals/sample:     973
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [217]:
@code_warntype sp.jointerms(p.terms, p.terms)

Variables:
  #self#::IJulia.sp.#jointerms
  t1::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1}
  t2::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1}
  j@_4::Int64
  #temp#@_5::Int64
  j@_6::Int64
  #temp#@_7::Int64
  terms::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1}
  i::Int64
  i1::Int64
  i2::Int64
  deletions::Int64
  #temp#@_13::Bool

Body:
  begin 
      SSAValue(9) = (Base.add_int)((Base.arraylen)(t1::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1})::Int64, (Base.arraylen)(t2::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1})::Int64)::Int64
      terms::Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1} = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1}, svec(Any, Int64), :($(QuoteNode(Array{IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}},1}))), 0, SSAValue(9), 0)) # line 169:
      i::Int64 = 1 # line 170:
      i1::Int64 = 1 # line 

In [209]:
@benchmark $x^2 + $y + $x * $x + 3 * $x * $y + $x * $y^2

BenchmarkTools.Trial: 
  memory estimate:  768 bytes
  allocs estimate:  8
  --------------
  minimum time:     276.022 ns (0.00% GC)
  median time:      309.009 ns (0.00% GC)
  mean time:        408.990 ns (17.10% GC)
  maximum time:     12.975 μs (90.53% GC)
  --------------
  samples:          10000
  evals/sample:     271
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [158]:
issorted(reverse(getfield.(p.terms, :monomial)))

false

In [157]:
getfield.(p.terms, :monomial)

4-element Array{IJulia.sp.Monomial{2,(:x, :y)},1}:
 x^2 
 y   
 xy  
 xy^2

In [136]:
@benchmark vcat($t1, convert(Vector, $t2))

BenchmarkTools.Trial: 
  memory estimate:  320 bytes
  allocs estimate:  4
  --------------
  minimum time:     203.301 ns (0.00% GC)
  median time:      226.203 ns (0.00% GC)
  mean time:        286.301 ns (14.39% GC)
  maximum time:     7.831 μs (91.69% GC)
  --------------
  samples:          10000
  evals/sample:     621
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [135]:
@benchmark vcat($t1, $t1)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  1
  --------------
  minimum time:     51.105 ns (0.00% GC)
  median time:      62.644 ns (0.00% GC)
  mean time:        79.123 ns (11.28% GC)
  maximum time:     2.142 μs (90.81% GC)
  --------------
  samples:          10000
  evals/sample:     986
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [49]:
m1 = sp.Monomial{2, (x, y)}((1, 0))
m2 = sp.Monomial{2, (x, y)}((0, 1))

y

In [50]:
Base.isless(m1, m2)

false

In [52]:
@benchmark $m1 < $m2

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.092 ns (0.00% GC)
  median time:      3.187 ns (0.00% GC)
  mean time:        3.512 ns (0.00% GC)
  maximum time:     20.902 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [53]:
@code_warntype Base.isless(m1, m2)

Variables:
  #self#::Base.#isless
  m1::IJulia.sp.Monomial{2,(x, y)}
  m2::IJulia.sp.Monomial{2,(x, y)}

Body:
  begin  # line 29:
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticvars.jl # line 39:
      SSAValue(0) = $(Expr(:invoke, MethodInstance for isless(::Tuple{Int64,Int64}, ::Tuple{Int64,Int64}), :(Base.isless), :((Core.getfield)(m1, :exponents)::Tuple{Int64,Int64}), :((Core.getfield)(m2, :exponents)::Tuple{Int64,Int64})))
      # meta: pop location
      return SSAValue(0)
  end::Bool


In [55]:
m3 = sp.Monomial{1, (x,)}((1,))

x

In [57]:
@benchmark $m1 < $m3

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.023 ns (0.00% GC)
  median time:      0.036 ns (0.00% GC)
  mean time:        0.041 ns (0.00% GC)
  maximum time:     8.205 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [45]:
(x, y) < (x, y, x)

true

In [34]:
@benchmark $x < $y

LoadError: [91mMethodError: no method matching isless(::IJulia.sp.Variable{:x}, ::IJulia.sp.Variable{:y})[39m

In [38]:
(x, y) < (y, x)

LoadError: [91mMethodError: no method matching isless(::IJulia.sp.Variable{:x}, ::IJulia.sp.Variable{:y})[39m

In [33]:
@which Base.isless(x, y)

LoadError: [91mno method found for the specified argument types[39m

In [30]:
sp.exponents(m1)

(1, 0)

In [25]:
@code_warntype Base.isless(m1, m2)

Variables:
  #self#::Base.#isless
  m1::IJulia.sp.Monomial{2,(x, y)}
  m2::IJulia.sp.Monomial{2,(x, y)}

Body:
  begin 
      SSAValue(0) = $(Expr(:invoke, MethodInstance for isless(::Tuple{IJulia.sp.Variable{:x},IJulia.sp.Variable{:y}}, ::Tuple{IJulia.sp.Variable{:x},IJulia.sp.Variable{:y}}), :(Base.isless), :($(QuoteNode((x, y)))), :($(QuoteNode((x, y))))))
      unless SSAValue(0) goto 4
      return SSAValue(0)
      4: 
      unless $(Expr(:invoke, MethodInstance for ==(::Tuple{IJulia.sp.Variable{:x},IJulia.sp.Variable{:y}}, ::Tuple{IJulia.sp.Variable{:x},IJulia.sp.Variable{:y}}), :(IJulia.sp.==), :($(QuoteNode((x, y)))), :($(QuoteNode((x, y)))))) goto 7
      return $(Expr(:invoke, MethodInstance for isless(::Tuple{Int64,Int64}, ::Tuple{Int64,Int64}), :(Base.isless), :((Core.getfield)(m1, :exponents)::Tuple{Int64,Int64}), :((Core.getfield)(m2, :exponents)::Tuple{Int64,Int64})))
      7: 
      return false
  end::Any
